In [8]:
import torch
import numpy as np

In [6]:
from config import Config
from data import Data
from channel import Channel

In [17]:
Nt = 1344
Na = 84
Nr = 73
Lin = 32
Lh = 6
config = Config(N_transmit_antenna=Nt, N_active_antenna=Na, N_receive_antenna=Nr, block_length=Lin, channel_length=Lh, batch=1, generator_mode='sparc', channel_truncation='tail', channel_profile='uniform')
data = Data(config)
channel = Channel(config)

In [26]:
b = data.generate_message()[0]
A = channel.generate_channel()
(A @ b).var()

tensor(1.1665, device='cuda:0')

In [2]:
import torch
x = torch.arange(10).view(2, 5)

In [6]:
x.sum(dim=-1, keepdim=True).repeat_interleave(5, dim=-1)

tensor([[10, 10, 10, 10, 10],
        [35, 35, 35, 35, 35]])

In [2]:
import torch
torch.exp(torch.tensor(1000000000000000000))

tensor(inf)

In [10]:
torch.tensor(1e-30)**2

tensor(0.)